In [1]:
import sys
sys.path.append('C:/Users/juans/OneDrive/Documentos/iaProyect/Cognitive')
print(sys.path)  # Esto mostrará todas las rutas en sys.path

from model.preprocessing.data_cleaning import DataCleaner
from model.utils.data_analysis import DataAnalyzer
from model.preprocessing.data_extraction import DataExtractor
from model.preprocessing.feature_extraction import FeatureSelector,FeatureVisualizer



['C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\python311.zip', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\DLLs', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\Lib', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0', 'c:\\Users\\juans\\OneDrive\\Documentos\\iaProyect\\Cognitive\\.venv', '', 'c:\\Users\\juans\\OneDrive\\Documentos\\iaProyect\\Cognitive\\.venv\\Lib\\site-packages', 'c:\\Users\\juans\\OneDrive\\Documentos\\iaProyect\\Cognitive\\.venv\\Lib\\site-packages\\win32', 'c:\\Users\\juans\\OneDrive\\Documentos\\iaProyect\\Cognitive\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\juans\\OneDrive\\Documentos\\iaProyect\\Cognitive\\.venv\\Lib\\site-packages\\Pythonwin', 'C:/Users/juans/OneDrive/Documentos/iaProyect/Cognitive']


In [2]:

# Ejemplo de uso
data_extractor = DataExtractor()

original_data = "/raw/DB sin RM (filtrado).csv" 
processed_data = "/DB sin RM (filtrado) - procesado.csv"

data_extractor.preprocess_csv(original_data, processed_data)
data = data_extractor.load_data("/processed", processed_data)


data_analyzer = DataAnalyzer()
#print(data_analyzer.describe_data(data=data))
#print features
print(data.columns.values)

['Identificacin' 'Lugar Nacimiento' 'Municipio Atencin' 'Edad' 'Gnero'
 'Grupo Sanguneo' 'Lateralidad' 'Autopercepcin Salud' 'Estado Civil'
 'Vive con pareja' 'Nmero Hijos' 'Tipo Vivienda'
 'Nmero Personas Viven Casa' 'Vive solo' 'Estrato' 'Escolaridad'
 'Ocupacin' 'Salario' 'Antecedentes Alcohol' 'Antecedentes Tabaco'
 'Antecedentes Sustancias Psicoactivas' 'Antecedentes Hta' 'MHTA-1'
 'MHTA-2' 'MHTA-3' 'Tiazida' 'Calcioantagonista' 'Betabloqueador' 'I-ECA'
 'ARA-II' 'D-ASA' 'AH-K' 'B-ALFA' 'B-ADRE' 'ASA'
 'Antecedentes Sindrome Coronario' 'Tipo SC' 'MSC-1' 'MSC-1.1' 'STENT'
 'NITRATOS' 'CLOPIDO' 'Antecedentes Arritmias Cardiacas' 'MARR-1' 'MARR-2'
 'AMIODARONA' 'ANTICOAG' 'DIGITAL' 'Antecedentes Acv' 'Tipo ACV'
 'Antecedentes Dislipidemia' 'Tipo dislipidemia' 'Tipo dislipidemia.1'
 'M-DISL' 'ESTATINA' 'FIBRATOS' 'Antecedentes EPOC' 'MEPOC' 'B2' 'ANTICOL'
 'GLUC-I' 'TEOFI' 'ANTILEUCO' 'OXIG' 'Antecedentes ERC' 'MERC-1' 'MERC-2'
 'MERC-3' 'CALCIO' 'A. FOLIC' 'COM. B' 'Antecedentes Canc

C:\Users/juans/OneDrive/Documentos/iaProyect/Cognitive\model\preprocessing\data_extraction.py:67: DtypeWarning: Columns (44,67) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path, encoding=encoding, delimiter=delimiter, header=0, quoting=csv.QUOTE_NONE, decimal=',')


In [ ]:


import pandas as pd
import numpy as np



# Instanciar y usar el preprocesador
preprocessor = DataCleaner(X, y)
data_analyzer = DataAnalyzer()


preprocessor.encode_features(target_column='income', method='onehot')

# Limpieza de duplicados y datos nulos
preprocessor.delete_null_data()
#preprocessor.delete_duplicate_rows()
#preprocessor.delete_Object_columns()

# Imputación de valores faltantes
#X_imputed = preprocessor.impute_missing_values()

# Estandarización
preprocessor.standardize()

# normalización
preprocessor.normalize_features()

#detectar y eliminar outliers
preprocessor.delete_outliers().value_counts()

#balancear datos
preprocessor.balance_data()

#generar columnas polinomiales
#xfold = preprocessor.generate_polynomial_columns()

#preprocessor.reduce_dimensionality(n_components=2,replace=True)

data = pd.concat([preprocessor.X, preprocessor.y], axis=1)
print(data_analyzer.describe_data(data=data))
data_analyzer.visualize_correlation(data=data)
data_analyzer.visualize_data(data, kind='boxplot')
data_analyzer.visualize_data(data, kind='histogram')




In [ ]:
X = preprocessor.X
y = preprocessor.y


# Inicializar el selector de características
selector = FeatureSelector(X, y)

# 1. Selección de características con Chi-squared
chi2_result = selector.select_features_chi2(k=14)
FeatureVisualizer.plot_scores(chi2_result['features'], chi2_result['scores'], title='Chi-squared Scores')

# 2. Selección de características con Spearman
spearman_result = selector.select_features_spearman(threshold=0.01)
FeatureVisualizer.plot_correlations(spearman_result['selected_features'], spearman_result['correlations'], title='Spearman Correlation Coefficients')

# 3. Selección de características con Lasso
lasso_result = selector.lasso_feature_selection(alpha=0.01)
FeatureVisualizer.plot_coefficients(lasso_result['selected_features'], lasso_result['coefficients'], title='Lasso Coefficients')

# 4. Selección de características con Sequential Feature Selector
sfs_result = selector.sequential_feature_selection(n_features_to_select=13, direction='forward')
print("Sequential Feature Selection Selected Features:", sfs_result['selected_features'])

# 5. Importancia de características con Random Forest
rf_result = selector.random_forest_feature_importance(n_features=14)
FeatureVisualizer.plot_importances(rf_result['selected_features'], rf_result['importances'], title='Random Forest Feature Importances')